# Yandex.Music

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>

En èste proyecto se analizaràn las preferencias musicales de las ciudades de Springfield y Shelbyville. Se estudiaràn datos reales de Yandex.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de ambas ciudades.

### Objetivo: 
Comprobar las hipòtesis: 
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad. 
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche. 
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.
 
[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Inicialmente se analizan los datos.

In [1]:
import pandas as pd # importando pandas


In [2]:
df= pd.read_csv( "/datasets/music_project_en.csv")# leyendo el archivo y almacenándolo en df
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,21:51:22,Friday
freq,76,136,136,8850,45360,14,23149


In [3]:
(df.head(10))# obteniendo las 10 primeras filas de la tabla df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Se obtiene la información general sobre la tabla:

In [4]:
print(df.info()) # obteniendo información general sobre los datos en df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB
None


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. Se identifican valores ausentes.

### Conclusiones <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista. 

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

[Volver a Contenidos](#back)

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

### Estilo del encabezado <a id='header_style'></a>
Se imprime el encabezado de las columnas:

In [5]:
print(df.columns)# la lista de los nombres de las columnas en la tabla df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Se cambian los nombres de algunas columnas.

In [29]:
df=df.rename(
    columns={
        "  userID":"user_id",
        "Track":"track",
        "  City  ":"city",
        "Day":"day"    
    }
)
        

Se comprueban nuevamente los nombres de las columnas.

In [30]:
(df.columns)# comprobando el resultado: la lista de los nombres de las columnas

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Volver a Contenidos](#back)

### Valores ausentes <a id='missing_values'></a>
Se identifican posibles valores ausentes.

In [31]:
df.isnull().sum()

user_id       0
track      1262
artist     7097
genre      1127
city          0
time          0
day           0
dtype: int64

Se identifican valores ausentes en las columnas "track","astist" y "genre". Los datos ausentes se rellenan con la string "unknown".

In [36]:
columns_to_replace= ["track","artist","genre"]
for value in columns_to_replace:
    df[value]=df[value].fillna("unknown")
    # recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'

Se verifica nuevamente la existencia de valores ausentes.

In [37]:
df.isna().sum()# contando valores ausentes

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Volver a Contenidos](#back)

### Duplicados <a id='duplicates'></a>
Se identifica posible existencia de datos ausentes.

In [38]:
df.duplicated().sum()# contando duplicado obvios

0

No se identifican datos duplicados.

Se imprime una lista de nombres únicos de géneros, ordenados en orden alfabético.

In [41]:
df["genre"].sort_values().unique()# inspeccionando los nombres de géneros únicos

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Se identifica en la lista los duplicados implícitos del género `hiphop`. Se encuentran los siguientes duplicados: hip, hop, hip-hop.

Se declara la función `replace_wrong_genres()` con dos parámetros: 
* `wrong_genres=` — la lista de duplicados
* `correct_genre=` — la string con el valor correcto.

In [42]:
def replace_wrong_genres(wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
        df["genre"]=df["genre"].replace(wrong_genre,correct_genre)
        
duplicates=["hip","hop","hip-hop"]
genre="hiphop"
                     

In [43]:
replace_wrong_genres(duplicates,genre)
print(df)


        user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
61248  729CBB09                            My Name            McLean   
61249  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
61250  C5E3A0D5                          Jalopiina           unknown   
61251  321D0506                      Freight Train     Chas McDevitt   
61252  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
1            rock  Springfi

Se imprime nuevamente la lista de valores únicos de la columna `'genre'`:

In [44]:
df["genre"].sort_values().unique()# revisando en busca de duplicados implícitos

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

[Volver a Contenidos](#back)

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

Se evalùa la actividad del usuario en cada ciudad. Se agrupan los datos por ciudad y encuentra el número de canciones reproducidas en cada grupo.

In [45]:
df.groupby("city")["user_id"].count()# contando las pistas reproducidas en cada ciudad

city
Shelbyville    18512
Springfield    42741
Name: user_id, dtype: int64

Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.

Se agrupan los datos por día de la semana y se encuentra el número de pistas reproducidas el lunes, miércoles y viernes.


In [46]:
df.groupby("day")["user_id"].count()# calculando las pistas reproducidas en cada uno de los tres días

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.


Se crea una funciòn para calcular el número de canciones reproducidas en un determinado día y ciudad. Requerirá dos parámetros:

Luego se calcula el nùmero de usuarios para cada dìa en cada Ciudad.

In [47]:

def number_tracks(day,city):
    
    track_list=df[(df["day"] == day)&(df["city"] == city)]
    track_list_count=track_list["user_id"].count()
    return track_list_count

Se recuperan los datos de ambas ciudades para cada uno de los tres días.

In [48]:
print(number_tracks("Monday","Springfield"))# el número de canciones reproducidas en Springfield el lunes

15740


In [49]:
print(number_tracks("Monday","Shelbyville"))# el número de canciones reproducidas en Shelbyville el lunes

5614


In [50]:
print(number_tracks("Wednesday","Springfield"))# el número de canciones reproducidas en Springfield el miércoles

11056


In [24]:
print(number_tracks("Wednesday","Shelbyville"))# el número de canciones reproducidas en Shelbyville el miércoles

7003


In [51]:
print(number_tracks("Friday","Springfield"))# el número de canciones reproducidas en Springfield el viernes

15945


In [52]:
print(number_tracks("Friday","Shelbyville"))# el número de canciones reproducidas en Shelbyville el viernes

5895


Se elabora una tabla que contenga los datos de la cantidad de usuarios por dìa en cada Ciudad.

In [53]:
datos=[
    ["Springfield",16715,11755,16890],
    ["Shelbyville",5982,7478,6259]
]

columnas=["city","Monday","Wednesday","Friday"]

tabla=pd.DataFrame(data=datos, columns=columnas)

In [54]:
print(tabla)# tabla con los resultados

          city  Monday  Wednesday  Friday
0  Springfield   16715      11755   16890
1  Shelbyville    5982       7478    6259


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

[Volver a Contenidos](#back)

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Se filtra la informaciòn para cada ciudad.

In [55]:
spr_general = df[df["city"]=="Springfield"]# obteniendo la tabla spr_general de las filas de df, 
print(spr_general)# donde los valores en la columna 'city' es 'Springfield'


        user_id                          track                   artist  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg   
4      E2DC1FAE                    Soul People               Space Echo   
6      4CB90AA5                           True             Roman Messer   
7      F03E1C1F               Feeling This Way          Polina Griffith   
8      8FA1D3BE                       L’estate              Julia Dalia   
...         ...                            ...                      ...   
61247  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder   
61248  729CBB09                        My Name                   McLean   
61250  C5E3A0D5                      Jalopiina                  unknown   
61251  321D0506                  Freight Train            Chas McDevitt   
61252  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez   

              genre         city      time        day  
1              rock  Springfield  14:07:09 

In [56]:
shel_general=df[df["city"]=="Shelbyville"]# obteniendo shel_general de las filas df,
print(shel_general)# donde el valor de la columna 'city' es 'Shelbyville'


        user_id                              track              artist  \
0      FFB692EC                  Kamigata To Boots    The Mass Missile   
2        20EC38                  Funiculì funiculà         Mario Lanza   
3      A3DD03C9              Dragons in the Sunset          Fire + Ice   
5      842029A1                             Chains            Obladaet   
9      E772D5C0                          Pessimist             unknown   
...         ...                                ...                 ...   
61239  D94F810B        Theme from the Walking Dead  Proyecto Halloween   
61240  BC8EC5CF       Red Lips: Gta (Rover Rework)               Rover   
61241  29E04611                       Bre Petrunko       Perunika Trio   
61242  1B91C621             (Hello) Cloud Mountain     sleepmakeswaves   
61249  D08D4A55  Maybe One Day (feat. Black Spade)         Blu & Exile   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
2  

Se elabora una funciòn que devuelva la informaciòn de los 15 gèneros mas populares de un dìa determinado en un periodo entre dos marcas de fecha y hora.
* Una tabla para los datos (`df`)
* El día de la semana (`day`)
* La marca de fecha y hora en formato 'hh:mm' (`time1`)
* La marca de fecha y hora en formato 'hh:mm' (`time2`)

In [68]:
def genre_weekday(df,day,time1,time2):
    genre_df = df[(df["day"] == day)&(df["time"]>time1)&(df["time"]<time2)]
    genre_df_grouped = genre_df.groupby("genre")["user_id"].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)

    return genre_df_sorted[:15]


Se comparan los resultados de la función `genre_weekday()`para Springfield y Shelbyville el lunes por la mañana (de 7 a 11) y el viernes por la tarde (de 17:00 a 23:00):

In [69]:
print(genre_weekday(spr_general,"Monday","07:00:00","11:00:00"))

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: user_id, dtype: int64


In [70]:
print(genre_weekday(shel_general,"Monday","07:00:00","11:00:00"))
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: user_id, dtype: int64


In [71]:
print(genre_weekday(spr_general,"Friday","17:00:00","23:00:00"))# llamando a la función para el viernes por la tarde en Springfield

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: user_id, dtype: int64


In [72]:
print(genre_weekday(shel_general,"Friday","17:00:00","23:00:00"))# llamando a la función para el viernes por la tarde en Shelbyville

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: user_id, dtype: int64


**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

[Volver a Contenidos](#back)

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

La tabla `spr_general` se agrupa por género y se encuentra el número de canciones reproducidas de cada género con el método `count()`. Después, se ordena el resultado en orden descendente.

In [73]:
spr_grouped = spr_general.groupby("genre")["genre"].count()
spr_genres= spr_grouped.sort_values(ascending=False)
print(spr_genres)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
               ... 
metalcore         1
marschmusik       1
malaysian         1
lovers            1
ïîï               1
Name: genre, Length: 250, dtype: int64


Se revisan las 10 primeras filas de `spr_genres`:

In [74]:
(spr_genres.head(10))# imprimiendo las 10 primeras filas de spr_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Se hace los mismo con los datos de Shelbyville.

In [75]:
shel_grouped = shel_general.groupby("genre")["genre"].count()
shel_genres= shel_grouped.sort_values(ascending=False)
print(shel_genres)

genre
pop           2431
dance         1932
rock          1879
electronic    1736
hiphop         960
              ... 
mandopop         1
leftfield        1
laiko            1
jungle           1
worldbeat        1
Name: genre, Length: 202, dtype: int64


Se imprimen las 10 primeras filas de `shel_genres`:

In [76]:
shel_genres.head(10)# imprimiendo las 10 primeras filas de shel_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades. 
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas. 

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.

[Volver a Contenidos](#back)